# 回顾

In [1]:
model="gpt-35-turbo"
import os
openai_api_base = os.environ["OPENAI_API_BASE_URL"];

In [2]:
from openai import OpenAI
import os
client = OpenAI(
    base_url = openai_api_base
)

In [3]:
def translate(text):
    messages = []
    messages.append( {"role": "system", 
                      "content": "You are a translator. Please, translate the user's request to English."})
    messages.append( {"role": "user", "content": text})
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.5,
        max_tokens = 100
    )
    print(response)
    return response.choices[0].message.content

In [5]:
import openai
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4")

print(encoding)

chinese = """在未来还没有到来的时候，总要有人把它创造出来，那个人应该是我们。"""
english = translate(chinese)

num_of_tokens_in_chinese = len(encoding.encode(chinese))
num_of_tokens_in_english = len(encoding.encode(english))

print(f"chinese:{chinese} ; {num_of_tokens_in_chinese} tokens\n")
print(f"english:{english} ; {num_of_tokens_in_english} tokens\n")

<Encoding 'cl100k_base'>
ChatCompletion(id='chatcmpl-8L6U7gju6GaAzXCr3gc4QV56OzAIf', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='"In the future, before it arrives, someone has to create it, and that person should be us."', role='assistant', function_call=None, tool_calls=None))], created=1700040507, model='gpt-35-turbo-16k', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=21, prompt_tokens=91, total_tokens=112))
chinese:在未来还没有到来的时候，总要有人把它创造出来，那个人应该是我们。 ; 35 tokens

english:"In the future, before it arrives, someone has to create it, and that person should be us." ; 21 tokens



# 常见应用场景

## 意图识别

In [11]:
import openai
response = client.chat.completions.create(
    model=model,
    temperature = 0,
    messages=[
        {"role": "system", "content": """
          Recognize the intent from the user's input 
         """},
        #{"role": "user", "content": "订明天早5点北京到上海的飞机"}
        {"role": "user", "content": "提醒我明早8点有会议"}
    ]
  )
print(response.choices[0].message.content)

Intent: Set Reminder


## 生成SQL

In [14]:
!pip3 install langchain  -i https://mirrors.aliyun.com/pypi/simple/

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [16]:
import openai, os
from langchain.llms import OpenAI

system_prompt =  """  You are a software engineer, you can anwser the user request based on the given tables:
                  table “students“ with the columns [id, name, course_id, score] 
                  table "courses" with the columns [id, name] 
                  """

prompt = system_prompt

response = client.chat.completions.create(
    model=model,
    temperature = 0,
    messages=[
        {"role": "system", "content": prompt},
        {"role": "user", "content": "计算所有学生英语课程的平均成绩"},
    ],
    max_tokens = 500
  )
print(response.choices[0].message.content)

To calculate the average score for all students in the English course, we need to join the "students" and "courses" tables on the "course_id" column and filter for students who are enrolled in English course.

Here is one way to write a SQL query to achieve this:

```
SELECT AVG(score) as average_score
FROM students
JOIN courses ON students.course_id = courses.id
WHERE courses.name = 'English';
```

Please note that this assumes the course name for English is exactly 'English'. If the course name is different in the "courses" table, you will need to adjust the query accordingly.


## 生成代码友好的提示词

### 规范输出的格式

In [17]:
import openai
response = client.chat.completions.create(
    model=model,
    temperature = 0,
    messages=[
        {"role": "system", "content": """
          Recognize the intent from the user's input and format output as JSON string. 
        The output JSON string includes: "intention", "paramters" """},
        {"role": "user", "content": "提醒我明早8点有会议"}
    ]
  )
print(response.choices[0].message.content)

{
  "intention": "reminder",
  "parameters": {
    "time": "8:00 AM",
    "event": "meeting",
    "date": "tomorrow"
  }
}


In [19]:
import openai, os
from langchain.llms import OpenAI

system_prompt =  """  You are a software engineer, you can write a SQL string as the anwser according to the user request 
               The user's requirement is based on the given tables:
                  table “students“ with the columns [id, name, course_id, score];
                  table "courses" with the columns [id, name]."""

prompt = system_prompt

response = client.chat.completions.create(
    model=model,
    temperature = 0,
    messages=[
        {"role": "system", "content": prompt},
        #{"role": "user", "content": "列出英语成绩大于80分的学生"},
        {"role": "user", "content": "列出英语课程成绩大于80分的学生, 返回结果只包括SQL"},
        #{"role": "user", "content": "列出年龄大于13的学生"}
    ],
    max_tokens = 500
  )
print(response.choices[0].message.content)

SELECT students.name
FROM students
INNER JOIN courses ON students.course_id = courses.id
WHERE courses.name = '英语' AND students.score > 80;


### 文本规范异常输出的格式 

In [20]:
import openai, os
from langchain.llms import OpenAI

system_prompt =  """  You are a software engineer, you can write a SQL string as the anwser according to the user request 
               The user's requirement is based on the given tables:
                  table “students“ with the columns [id, name, course_id, score];
                  table "courses" with the columns [id, name]."""

system_prompt_with_negative =  """  
You are a software engineer, you can write a SQL string as the anwser according to the user request.
Also, when you cannot create the SQL query for the user's request based on the given tables, please, only return "invalid request"
               The user's requirement is based on the given tables:
                  table “students“ with the columns [id, name, course_id, score];
                  table "courses" with the columns [id, name]."""

#prompt = system_prompt
prompt = system_prompt_with_negative

response = client.chat.completions.create(
    model=model,
    temperature = 0,
    messages=[
        {"role": "system", "content": prompt},
        {"role": "user", "content": "列出年龄大于13的学生"}
    ],
    max_tokens = 500
  )
print(response.choices[0].message.content)

SELECT * FROM students WHERE age > 13;
